<a href="https://colab.research.google.com/github/Terence0408/Teach_code/blob/master/Deep_Learning_R_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Directly use R environment
1. Click `Edit` \ `編輯`
2. Click `Notebook settings` \ `筆記本設定`
3. At `runtime type` \ `執行階段類型`: select R

In [7]:
version

               _                           
platform       x86_64-pc-linux-gnu         
arch           x86_64                      
os             linux-gnu                   
system         x86_64, linux-gnu           
status                                     
major          4                           
minor          3.2                         
year           2023                        
month          10                          
day            31                          
svn rev        85441                       
language       R                           
version.string R version 4.3.2 (2023-10-31)
nickname       Eye Holes                   

# Load package
Tutorial: https://tensorflow.rstudio.com/tutorials/

In [8]:
install.packages("keras")
install.packages('reticulate') # reticulate: Reticulate: R Interface to Python
install.packages('ramify')
install.packages('word2vec')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [9]:
library(keras)
library(reticulate)
library(tidyverse)
library(ramify)
library(word2vec)

# Word Embeddings

## Train by yourself

## Use trained

# RNN
Recurrent Neural Networks

https://github.com/rstudio/keras/blob/main/vignettes/examples/imdb_lstm.R

## Data

In [10]:
max_features = 20000
# cut texts after this number of words (among top max_features most common words)

# Loading data
imdb = dataset_imdb(num_words = max_features)

X_train <- imdb$train$x
y_train <- imdb$train$y
X_test <- imdb$test$x
y_test <- imdb$test$y

cat(length(X_train), 'train sequences\n')
cat(length(X_test), 'test sequences\n')

25000 train sequences
25000 test sequences


In [11]:
X_train <- pad_sequences(X_train, maxlen = 80)
X_test <- pad_sequences(X_test, maxlen = 80)

cat('x_train shape:', dim(X_train), '\n')
X_train[1,]

x_train shape: 25000 80 


[1]    15   256     4     2     7  3766     5   723    36    71    43   530
[13]   476    26   400   317    46     7     4 12118  1029    13   104    88
[25]     4   381    15   297    98    32  2071    56    26   141     6   194
[37]  7486    18     4   226    22    21   134   476    26   480     5   144
[49]    30  5535    18    51    36    28   224    92    25   104     4   226
[61]    65    16    38  1334    88    12    16   283     5    16  4472   113
[73]   103    32    15    16  5345    19   178    32

In [12]:
y_train[1]

[1] 1

## Model

### RNN

In [13]:
# Build the Keras Sequential model
RNN_model <- keras_model_sequential()

RNN_model %>%
  layer_embedding(input_dim = max_features, output_dim = 128) %>%
  layer_simple_rnn(128, dropout=0.2, recurrent_dropout=0.2, return_sequences= FALSE) %>%
  layer_dense(units = 1, activation = 'sigmoid')


# compile model
RNN_model %>% compile(
  loss = 'binary_crossentropy', #some other options: sparse_categorical_crossentropy
  optimizer = optimizer_adam(), #or: SGD, RMSProp
  metrics = c('accuracy'))

summary(RNN_model)

Model: "sequential"
________________________________________________________________________________
 Layer (type)                       Output Shape                    Param #     
 embedding (Embedding)              (None, None, 128)               2560000     
 simple_rnn (SimpleRNN)             (None, 128)                     32896       
 dense (Dense)                      (None, 1)                       129         
Total params: 2593025 (9.89 MB)
Trainable params: 2593025 (9.89 MB)
Non-trainable params: 0 (0.00 Byte)
________________________________________________________________________________


In [14]:
# Evalute the model
start_time = Sys.time() # Record the start time

RNN_model %>% fit(
    X_train, y_train,
    batch_size = 32,
    epochs = 1,
    verbose = 1,
    validation_data = list(X_test, y_test))

# Record the run time
end_time = Sys.time()
RNN_time = end_time - start_time

RNN_result <- RNN_model %>% evaluate(X_test, y_test,batch_size = 32)
RNN_score = RNN_result[1]
RNN_acc = RNN_result[2]

print(paste('Test score:', RNN_score))
print(paste('Test accuracy:', RNN_acc))
print(paste('Duration: ', RNN_time))

[1] "Test score: 0.640228986740112"
[1] "Test accuracy: 0.596920013427734"
[1] "Duration:  2.39325828552246"


### LSTM

In [15]:
# Build the Keras Sequential model
LSTM_model <- keras_model_sequential()

LSTM_model %>%
  layer_embedding(input_dim = max_features, output_dim = 128) %>%
  layer_lstm(128, dropout=0.2, recurrent_dropout=0.2, return_sequences= FALSE) %>%
  layer_dense(units = 1, activation = 'sigmoid')


# compile model
LSTM_model %>% compile(
  loss = 'binary_crossentropy', #some other options: sparse_categorical_crossentropy
  optimizer = optimizer_adam(), #or: SGD, RMSProp
  metrics = c('accuracy'))

summary(LSTM_model)

Model: "sequential_1"
________________________________________________________________________________
 Layer (type)                       Output Shape                    Param #     
 embedding_1 (Embedding)            (None, None, 128)               2560000     
 lstm (LSTM)                        (None, 128)                     131584      
 dense_1 (Dense)                    (None, 1)                       129         
Total params: 2691713 (10.27 MB)
Trainable params: 2691713 (10.27 MB)
Non-trainable params: 0 (0.00 Byte)
________________________________________________________________________________


In [16]:
# Evalute the model
start_time = Sys.time() # Record the start time

LSTM_model %>% fit(
    X_train, y_train,
    batch_size = 32,
    epochs = 1,
    verbose=1,
    validation_data = list(X_test, y_test))

# Record the run time
end_time = Sys.time()
LSTM_time = end_time - start_time

LSTM_result <- LSTM_model %>% evaluate(X_test, y_test,batch_size = 32)
LSTM_score = LSTM_result[1]
LSTM_acc = LSTM_result[2]

print(paste('Test score:', LSTM_score))
print(paste('Test accuracy:', LSTM_acc))
print(paste('Duration: ', LSTM_time))

[1] "Test score: 0.364283174276352"
[1] "Test accuracy: 0.836960017681122"
[1] "Duration:  5.33190276225408"


### GRU

In [17]:
# Build the Keras Sequential model
GRU_model <- keras_model_sequential()

GRU_model %>%
  layer_embedding(input_dim = max_features, output_dim = 128) %>%
  layer_gru(128, dropout=0.2, recurrent_dropout=0.2, return_sequences= FALSE) %>%
  layer_dense(units = 1, activation = 'sigmoid')


# compile model
GRU_model %>% compile(
  loss = 'binary_crossentropy', #some other options: sparse_categorical_crossentropy
  optimizer = optimizer_adam(), #or: SGD, RMSProp
  metrics = c('accuracy'))

summary(GRU_model)

Model: "sequential_2"
________________________________________________________________________________
 Layer (type)                       Output Shape                    Param #     
 embedding_2 (Embedding)            (None, None, 128)               2560000     
 gru (GRU)                          (None, 128)                     99072       
 dense_2 (Dense)                    (None, 1)                       129         
Total params: 2659201 (10.14 MB)
Trainable params: 2659201 (10.14 MB)
Non-trainable params: 0 (0.00 Byte)
________________________________________________________________________________


In [18]:
# Evalute the model
start_time = Sys.time() # Record the start time

GRU_model %>% fit(
    X_train, y_train,
    batch_size = 32,
    epochs = 1,
    verbose=1,
    validation_data = list(X_test, y_test))

# Record the run time
end_time = Sys.time()
GRU_time = end_time - start_time

GRU_result <- GRU_model %>% evaluate(X_test, y_test,batch_size = 32)
GRU_score = GRU_result[1]
GRU_acc = GRU_result[2]

print(paste('Test score:', GRU_score))
print(paste('Test accuracy:', GRU_acc))
print(paste('Duration: ', GRU_time))

[1] "Test score: 0.362203389406204"
[1] "Test accuracy: 0.839359998703003"
[1] "Duration:  5.41634857257207"


### biLSTM

In [19]:
# Build the Keras Sequential model
biLSTM_model <- keras_model_sequential()

biLSTM_model %>%
  layer_embedding(input_dim = max_features, output_dim = 128) %>%
  bidirectional(layer_lstm(units = 128, dropout=0.2, recurrent_dropout=0.2, return_sequences= FALSE)) %>%
  layer_dense(units = 1, activation = 'sigmoid')


# compile model
biLSTM_model %>% compile(
  loss = 'binary_crossentropy', #some other options: sparse_categorical_crossentropy
  optimizer = optimizer_adam(), #or: SGD, RMSProp
  metrics = c('accuracy'))

summary(biLSTM_model)

Model: "sequential_3"
________________________________________________________________________________
 Layer (type)                       Output Shape                    Param #     
 embedding_3 (Embedding)            (None, None, 128)               2560000     
 bidirectional (Bidirectional)      (None, 256)                     263168      
 dense_3 (Dense)                    (None, 1)                       257         
Total params: 2823425 (10.77 MB)
Trainable params: 2823425 (10.77 MB)
Non-trainable params: 0 (0.00 Byte)
________________________________________________________________________________


In [20]:
# Evalute the model
start_time = Sys.time() # Record the start time

biLSTM_model %>% fit(
    X_train, y_train,
    batch_size = 32,
    epochs = 1,
    verbose=1,
    validation_data = list(X_test, y_test))

# Record the run time
end_time = Sys.time()
biLSTM_time = end_time - start_time

biLSTM_result <- biLSTM_model %>% evaluate(X_test, y_test,batch_size = 32)
biLSTM_score = biLSTM_result[1]
biLSTM_acc = biLSTM_result[2]

print(paste('Test score:', biLSTM_score))
print(paste('Test accuracy:', biLSTM_acc))
print(paste('Duration: ', biLSTM_time))

[1] "Test score: 0.359871864318848"
[1] "Test accuracy: 0.843400001525879"
[1] "Duration:  10.2673135956128"


### biLSTM_custom

In [21]:
# Build the Keras Sequential model
biLSTM_custom_model <- keras_model_sequential()

forward_layer = layer_lstm(units = 128, dropout=0.2, recurrent_dropout=0.2, return_sequences= FALSE)
backward_layer = layer_lstm(units = 128, dropout=0.2, recurrent_dropout=0.2, return_sequences= FALSE, go_backwards=TRUE)

biLSTM_custom_model %>%
  layer_embedding(input_dim = max_features, output_dim = 128) %>%
  bidirectional(forward_layer, backward_layer=backward_layer) %>%
  layer_dense(units = 1, activation = 'sigmoid')


# compile model
biLSTM_custom_model %>% compile(
  loss = 'binary_crossentropy', #some other options: sparse_categorical_crossentropy
  optimizer = optimizer_adam(), #or: SGD, RMSProp
  metrics = c('accuracy'))

summary(biLSTM_custom_model)

Model: "sequential_4"
________________________________________________________________________________
 Layer (type)                       Output Shape                    Param #     
 embedding_4 (Embedding)            (None, None, 128)               2560000     
 bidirectional_1 (Bidirectional)    (None, 256)                     263168      
 dense_4 (Dense)                    (None, 1)                       257         
Total params: 2823425 (10.77 MB)
Trainable params: 2823425 (10.77 MB)
Non-trainable params: 0 (0.00 Byte)
________________________________________________________________________________


In [22]:
# Evalute the model
start_time = Sys.time() # Record the start time

biLSTM_custom_model %>% fit(
    X_train, y_train,
    batch_size = 32,
    epochs = 1,
    verbose=1,
    validation_data = list(X_test, y_test))

# Record the run time
end_time = Sys.time()
biLSTM_custom_time = end_time - start_time

biLSTM_custom_result <- biLSTM_custom_model %>% evaluate(X_test, y_test,batch_size = 32)
biLSTM_custom_score = biLSTM_result[1]
biLSTM_custom_acc = biLSTM_result[2]

print(paste('Test score:', biLSTM_custom_score))
print(paste('Test accuracy:', biLSTM_custom_acc))
print(paste('Duration: ', biLSTM_custom_time))

[1] "Test score: 0.359871864318848"
[1] "Test accuracy: 0.843400001525879"
[1] "Duration:  10.4809190988541"


## Compare

In [23]:
print(paste('RNN Test accuracy:', RNN_acc))
print(paste('LSTM Test accuracy:', LSTM_acc))
print(paste('GRU Test accuracy:', GRU_acc))
print(paste('biLSTM Test accuracy:', biLSTM_acc))
print(paste('biLSTM_custom Test accuracy:', biLSTM_custom_acc))

[1] "RNN Test accuracy: 0.596920013427734"
[1] "LSTM Test accuracy: 0.836960017681122"
[1] "GRU Test accuracy: 0.839359998703003"
[1] "biLSTM Test accuracy: 0.843400001525879"
[1] "biLSTM_custom Test accuracy: 0.843400001525879"


In [24]:
print(paste('RNN Test accuracy:', RNN_time))
print(paste('LSTM Test accuracy:', LSTM_time))
print(paste('GRU Test accuracy:', GRU_time))
print(paste('biLSTM Test accuracy:', biLSTM_time))
print(paste('biLSTM_custom Test accuracy:', biLSTM_custom_time))

[1] "RNN Test accuracy: 2.39325828552246"
[1] "LSTM Test accuracy: 5.33190276225408"
[1] "GRU Test accuracy: 5.41634857257207"
[1] "biLSTM Test accuracy: 10.2673135956128"
[1] "biLSTM_custom Test accuracy: 10.4809190988541"
